In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import matplotlib

import cv2

from skimage.morphology import watershed
from skimage.feature import peak_local_max
from PIL import Image

from skimage.morphology import skeletonize, thin
from skimage.util import invert
from skimage import measure
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import black_tophat, skeletonize, convex_hull_image
from skimage.morphology import disk
from skimage import feature
from skimage.filters import roberts, sobel, scharr, prewitt
import seaborn as sns
sns.reset_orig()

from skimage import restoration

from skimage import exposure
from skimage.filters import gaussian

from scipy import ndimage

from time import time

from skimage.color import rgb2gray

from skimage.color import rgb2hsv
from skimage.color import hsv2rgb



from skimage.filters.rank import entropy
import scipy

from skimage import data, filters
import scipy

import networkx as nx
from scipy import interpolate

from skimage.segmentation import random_walker
from skimage.data import binary_blobs
from skimage.exposure import rescale_intensity
import glob

from skimage.morphology import disk
from skimage import io

from mpl_toolkits.mplot3d import Axes3D
from skimage import io
from skimage import data
from skimage.filters import gaussian
from skimage.segmentation import active_contour

/Users/spencerlab/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)


In [2]:
parade = np.asarray(Image.open('RGBParadeImagesSpencerLab/RedChannelParadeRGBSequence1103.tif'))


mask= np.logical_and((parade[:,:,1]==parade[:,:,0]), (parade[:,:,2]==parade[:,:,0]))
parade2 = parade.copy()

mask=mask==0
parade2[:,:,0] = mask*parade[:,:,0]
parade2[:,:,1] = mask*parade[:,:,1]
parade2[:,:,2] = mask*parade[:,:,2]
parade2[:,:,3] = mask*parade[:,:,3]

In [3]:
stack = io.imread("Stitch_9_G001.tif")

In [6]:
def hsv_constraints(image):
    
    hsv_img = rgb2hsv(image)
    
    r_flat = (image[:,:,0]).flatten()
    g_flat = (image[:,:,1]).flatten()
    b_flat = (image[:,:,2]).flatten()



    h_flat = (hsv_img[:,:,0]).flatten()
    s_flat = (hsv_img[:,:,1]).flatten()
    v_flat = (hsv_img[:,:,2]).flatten()
    
    
    
    h_flat_2 = []
    s_flat_2 = []
    v_flat_2 = []
    
    
    for i in range(len(r_flat)):
        if r_flat[i]!=0 or g_flat[i]!=0 or b_flat[i]!=0:
            h_flat_2.append(h_flat[i])
            s_flat_2.append(s_flat[i])
            v_flat_2.append(v_flat[i])
    
    
    h_flat_2 = np.asarray(h_flat_2)

    
    h_unk = np.unique(h_flat_2)
    min_hue = h_unk[0]
    max_hue = h_unk[-1]


    diff = max_hue-min_hue
    
    
    
    
    
    
    
    neg = (h_flat_2-1)*(h_flat_2>=0.5)
    pos = h_flat_2*(h_flat_2<0.5)
    rang = neg+pos
    unk_rang = np.unique(rang)
    
    min_hue_2 = unk_rang[0]
    max_hue_2 = unk_rang[-1]
    
    
    if diff> max_hue_2-min_hue_2:
        min_hue = min_hue_2
        max_hue = max_hue_2
    
    
    
    
    
    
    
    
    unk_Sat = np.unique(s_flat_2)
    unk_Val = np.unique(v_flat_2)
    
    
    min_sat = unk_Sat[0]
    max_sat = unk_Sat[-1]
    
    min_val = unk_Val[0]
    max_val = unk_Val[-1]
    
    return min_hue, max_hue, min_sat, max_sat, min_val, max_val

In [7]:
def keep_inrange(hsv_img, min_h, max_h):
    
    
    neg = (hsv_img[:,:,0]-1)*(hsv_img[:,:,0]>=0.5)
    #neg=neg*(neg>=min_h)
    
    
    pos = hsv_img[:,:,0]*(hsv_img[:,:,0]<0.5)
    #pos = pos*(pos<=max_h)
    
    
    #ret = neg+pos
    
    ret_hsv = hsv_img.copy()
    
    ret_hsv[:,:,1] = hsv_img[:,:,1]*(neg>=min_h)*(pos<=max_h)
    
    return ret_hsv

In [8]:
def clean(inpt):
    ret = inpt.copy()
    for i in range(len(inpt)):
        for x in range(len(inpt[0])):
            if np.isnan(inpt[i,x]):
                ret[i,x]=0
            else:
                ret[i,x]=inpt[i,x]
        
    return ret

In [9]:
def denoise_repeat(image_stk, patch_size, patch_distance, h, n):
    ret = image_stk
    for i in range(n):
        ret=restoration.denoise_nl_means(ret, patch_size=patch_size, patch_distance=patch_distance, h=h)
        ret=clean(ret)
    return ret

In [10]:
def mix_filters(image, patch_size, patch_distance, h, n, canny_sigma, denoise_tresh):
    nlm = denoise_repeat(image[:,:,0], patch_size, patch_distance, h, n)
    
    edges = feature.canny(image[:,:,0], canny_sigma)
    
    
    mix_tresh = np.logical_or(edges,(nlm>denoise_tresh))
    
    return mix_tresh

In [11]:
def remove_specs(inpt, max_spec_size):
    ret = np.ones(inpt.shape)
    for i in range(len(inpt)):
        for x in range(len(inpt[0])):
            num = 0
            for n1 in range(-1,2):
                for n2 in range(-1,2):
                    try:
                        if inpt[i+n1, x+n2] ==1:
                            num=num+1
                    except:
                        cat = 0
            if num <=max_spec_size:
                ret[i, x] = 0
            else:
                ret[i, x] = inpt[i, x]
            
    return ret

In [12]:
mih, mah, mis, mas, miv, mav = hsv_constraints(parade2[:,:,:3])

/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/color/colorconv.py:279: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 2. + (arr[idx, 2] - arr[idx, 0]) / delta[idx]
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/color/colorconv.py:283: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = 4. + (arr[idx, 0] - arr[idx, 1]) / delta[idx]


In [16]:
[mih, mah, mis, mas, miv, mav]

[-0.03, 0.06047197640117993, 0.6, 1.0, 0.00784313725490196, 1.0]

In [14]:
mis = 0.6
mih = -0.03

In [17]:
def color_leave_filter(image, min_hue, max_hue, min_sat, max_sat, min_val, max_val):
    
    hsv_img = rgb2hsv(image) 
    
    kir = keep_inrange(hsv_img, min_hue, max_hue)
    
    kir2=kir.copy()
    kir2[:,:,1]=kir2[:,:,1]
    kir2[:,:,2]=kir2[:,:,2]*(kir2[:,:,1]>=min_sat)*(kir2[:,:,1]<=max_sat)*(kir2[:,:,2]>=min_val)*(kir2[:,:,2]<=max_val)
    

    return (np.round(hsv2rgb(kir2)*255).astype('uint8'))

In [ ]:
for i in range(180, len(stack)):
    clv = color_leave_filter(stack[i], mih, mah, mis, mas, miv, mav)
    mfilt = mix_filters(image=clv, patch_size=450, patch_distance=9, h=0.1, n=2, canny_sigma=2, denoise_tresh=50)

    square = mfilt!=0
    
    eroded_square = ndimage.binary_erosion(square)
    reconstruction = ndimage.binary_propagation(eroded_square, mask=square)
    reconstruction2 = ndimage.binary_closing(reconstruction)
    
    print(i)
    
    Image.fromarray(((reconstruction2!=0)*255).astype('uint8')).save('blood_vessels_day_3_v3/'+str(i)+'.tif')

/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/color/colorconv.py:269: RuntimeWarning: divide by zero encountered in true_divide
  out_s = delta / out_v
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/color/colorconv.py:275: RuntimeWarning: divide by zero encountered in true_divide
  out[idx, 0] = (arr[idx, 1] - arr[idx, 2]) / delta[idx]
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/restoration/non_local_means.py:132: UserWarning: denoise_nl_means will default to multichannel=False in v0.15
  warn('denoise_nl_means will default to multichannel=False in v0.15')
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/restoration/non_local_means.py:132: UserWarning: denoise_nl_means will default to multichannel=False in v0.15
  warn('denoise_nl_means will default to multichannel=False in v0.15')


180


/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/restoration/non_local_means.py:132: UserWarning: denoise_nl_means will default to multichannel=False in v0.15
  warn('denoise_nl_means will default to multichannel=False in v0.15')
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/restoration/non_local_means.py:132: UserWarning: denoise_nl_means will default to multichannel=False in v0.15
  warn('denoise_nl_means will default to multichannel=False in v0.15')


181


/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/restoration/non_local_means.py:132: UserWarning: denoise_nl_means will default to multichannel=False in v0.15
  warn('denoise_nl_means will default to multichannel=False in v0.15')
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/restoration/non_local_means.py:132: UserWarning: denoise_nl_means will default to multichannel=False in v0.15
  warn('denoise_nl_means will default to multichannel=False in v0.15')


...

245


/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/restoration/non_local_means.py:132: UserWarning: denoise_nl_means will default to multichannel=False in v0.15
  warn('denoise_nl_means will default to multichannel=False in v0.15')
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/restoration/non_local_means.py:132: UserWarning: denoise_nl_means will default to multichannel=False in v0.15
  warn('denoise_nl_means will default to multichannel=False in v0.15')


246


/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/restoration/non_local_means.py:132: UserWarning: denoise_nl_means will default to multichannel=False in v0.15
  warn('denoise_nl_means will default to multichannel=False in v0.15')
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/skimage/restoration/non_local_means.py:132: UserWarning: denoise_nl_means will default to multichannel=False in v0.15
  warn('denoise_nl_means will default to multichannel=False in v0.15')


247
